# 라이브러리

In [2]:
options(warn =-1, repr.plot.width=15, repr.plot.height=9)
# ## 패키지 설치
library(dplyr)
library(stringr)
library(ggplot2)
library(gridExtra)
library(scales)
library(randomForest)
library(lubridate)
library(caret)
library(e1071)
library(nnet)
library(class)



다음의 패키지를 부착합니다: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



다음의 패키지를 부착합니다: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


다음의 패키지를 부착합니다: ‘randomForest’


The following object is masked from ‘package:gridExtra’:

    combine


The following object is masked from ‘package:ggplot2’:

    margin


The following object is masked from ‘package:dplyr’:

    combine



다음의 패키지를 부착합니다: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


필요한 패키지를 로딩중입니다: lattice



# 전처리

In [3]:
### .csv 불러오기
data <- read.csv("./data/refined_data_filtered_major5_subset.csv")
data1 <- read.csv("./data/refined_data_filtered_major5_subset_2021.csv")
data<- rbind(data,data1)

In [4]:
data$근무지역분류 <- as.character(data$근무지역분류)
data$근무지역분류 <- factor(data$근무지역분류)
data$장애유형 <- as.character(data$장애유형)
data$장애유형 <- factor(data$장애유형)
data$중증여부 <- as.character(data$중증여부)
data$중증여부 <- factor(data$중증여부)
data$근무지역 <- as.character(data$근무지역)
data$근무지역 <- factor(data$근무지역)
data$취업직종대분류 <- as.character(data$취업직종대분류)
data$취업직종대분류 <- factor(data$취업직종대분류)
data$연령대 <- as.character(data$연령대)
data$연령대 <- factor(data$연령대)
data$장애유형_중증여부 <- as.character(data$장애유형_중증여부)
data$장애유형_중증여부 <- factor(data$장애유형_중증여부)
data$월 <- as.character(data$월)
data$월 <- factor(data$월)

In [5]:
data$target<-(ifelse(data$취업직종대분류=="경영·행정·사무직","경영·행정·사무직",data$취업직종대분류))

data$target <- ifelse(data$취업직종대분류 %in% c("청소 및 기타 개인서비스직", "음식 서비스직", "미용·예식 서비스직", "영업·판매직", "스포츠·레크리에이션직","경호·경비직"), "서비스직", data$target)

data$target <- ifelse(grepl("^\\d+$", data$target), "기타", data$target)

In [6]:
data <- subset(data, data$target == "서비스직")
table(data$target)


서비스직 
   18912 

In [7]:
data$target <- ifelse(data$취업직종대분류 %in% c("청소 및 기타 개인서비스직"), "청소 및 기타 개인서비스직", "서비스 및 판매 관련 직군")

In [8]:
table(data$target)


 서비스 및 판매 관련 직군 청소 및 기타 개인서비스직 
                     6231                     12681 

In [9]:
# 불필요한 컬럼 제거

data_filtered <- subset(data, select = -c(취업일자,연령,장애유형,중증여부,근무지역))
sum(is.na(data_filtered))

[1] 0

In [10]:
# 연령대
data_filtered$age_20 <- data_filtered$연령대 == 1
data_filtered$age_30 <- data_filtered$연령대 == 2
data_filtered$age_40 <- data_filtered$연령대 == 3
data_filtered$age_50 <- data_filtered$연령대 == 4
data_filtered$age_60 <- data_filtered$연령대 == 5
data_filtered$age_70 <- data_filtered$연령대 == 6

In [11]:
# 장애유형_중증여부 바꾸기
data_filtered$시각장애_중증 <- data_filtered$장애유형_중증여부 == "시각장애-중증"
data_filtered$시각장애_경증 <- data_filtered$장애유형_중증여부 == "시각장애-경증"

data_filtered$지체장애_중증 <- data_filtered$장애유형_중증여부 == "지체장애-중증"
data_filtered$지체장애_경증 <- data_filtered$장애유형_중증여부 == "지체장애-경증"

data_filtered$지적장애_경증 <- data_filtered$장애유형_중증여부 == "지적장애-경증"
data_filtered$지적장애_중증 <- data_filtered$장애유형_중증여부 == "지적장애-중증"

data_filtered$청각장애_경증 <- data_filtered$장애유형_중증여부 == "청각장애-경증"
data_filtered$청각장애_중증 <- data_filtered$장애유형_중증여부 == "청각장애-중증"

data_filtered$정신장애_경증 <- data_filtered$장애유형_중증여부 == "정신장애-경증"
data_filtered$정신장애_중증 <- data_filtered$장애유형_중증여부 == "정신장애-중증"

In [12]:
#   근무지역 분류 바꾸기
data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

data_filtered$강원 <- data_filtered$근무지역분류 == "강원"

data_filtered$전남 <- data_filtered$근무지역분류 == "전남"

data_filtered$충북 <- data_filtered$근무지역분류 == "충북"

data_filtered$경기 <- data_filtered$근무지역분류 == "경기"

data_filtered$인천 <- data_filtered$근무지역분류 == "인천"

data_filtered$서울 <- data_filtered$근무지역분류 == "서울"

data_filtered$울산 <- data_filtered$근무지역분류 == "울산"

data_filtered$대전 <- data_filtered$근무지역분류 == "대전"

data_filtered$부산 <- data_filtered$근무지역분류 == "부산"

data_filtered$전북 <- data_filtered$근무지역분류 == "전북"

data_filtered$광주 <- data_filtered$근무지역분류 == "광주"

data_filtered$충남 <- data_filtered$근무지역분류 == "충남"

data_filtered$대구 <- data_filtered$근무지역분류 == "대구"

data_filtered$경남 <- data_filtered$근무지역분류 == "경남"

data_filtered$제주 <- data_filtered$근무지역분류 == "제주"

data_filtered$세종 <- data_filtered$근무지역분류 == "세종"

data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

In [13]:
# 월
data_filtered$Jan <- data_filtered$월  == 01
data_filtered$Feb <- data_filtered$월  == 02
data_filtered$Mar <- data_filtered$월  == 03
data_filtered$Apr <- data_filtered$월  == 04
data_filtered$May <- data_filtered$월  == 05
data_filtered$Jun <- data_filtered$월  == 06
data_filtered$Jul <- data_filtered$월  == 07
data_filtered$Aug <- data_filtered$월  == 08
data_filtered$Sep <- data_filtered$월  == 09
data_filtered$Oct <- data_filtered$월  == 10
data_filtered$Nov <- data_filtered$월  == 11
data_filtered$Dec <- data_filtered$월  == 12

In [14]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	18912 obs. of  51 variables:
 $ 취업직종대분류   : Factor w/ 23 levels "건설·채굴직",..: 22 22 22 22 22 22 22 22 22 22 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 4 1 2 1 5 1 6 3 3 2 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 7 6 6 6 8 6 2 6 4 6 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 9 1 1 1 1 1 1 1 1 1 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 13 17 17 17 17 17 17 17 17 17 ...
 $ target           : chr  "청소 및 기타 개인서비스직" "청소 및 기타 개인서비스직" "청소 및 기타 개인서비스직" "청소 및 기타 개인서비스직" ...
 $ age_20           : logi  FALSE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30           : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50           : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  FALSE FALSE FALSE FALSE FALSE F

[1] 0

In [15]:
str(data_filtered)
sum(is.na(data_filtered))

'data.frame':	18912 obs. of  51 variables:
 $ 취업직종대분류   : Factor w/ 23 levels "건설·채굴직",..: 22 22 22 22 22 22 22 22 22 22 ...
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 4 1 2 1 5 1 6 3 3 2 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 7 6 6 6 8 6 2 6 4 6 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 9 1 1 1 1 1 1 1 1 1 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 13 17 17 17 17 17 17 17 17 17 ...
 $ target           : chr  "청소 및 기타 개인서비스직" "청소 및 기타 개인서비스직" "청소 및 기타 개인서비스직" "청소 및 기타 개인서비스직" ...
 $ age_20           : logi  FALSE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30           : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50           : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60           : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ age_70           : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증    : logi  FALSE FALSE FALSE FALSE FALSE F

[1] 0

In [16]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [17]:
str(data_filtered_subset)
sum(is.na(data_filtered_subset))

'data.frame':	18912 obs. of  47 variables:
 $ 취업직종대분류: Factor w/ 23 levels "건설·채굴직",..: 22 22 22 22 22 22 22 22 22 22 ...
 $ target        : chr  "청소 및 기타 개인서비스직" "청소 및 기타 개인서비스직" "청소 및 기타 개인서비스직" "청소 및 기타 개인서비스직" ...
 $ age_20        : logi  FALSE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30        : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50        : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ 지체장애_경증 : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  FALSE TRUE TRUE TRUE FALSE TRUE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE F

[1] 0

In [18]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

In [19]:
str(data_filtered_subset)

'data.frame':	18912 obs. of  47 variables:
 $ 취업직종대분류: Factor w/ 23 levels "건설·채굴직",..: 22 22 22 22 22 22 22 22 22 22 ...
 $ target        : chr  "청소 및 기타 개인서비스직" "청소 및 기타 개인서비스직" "청소 및 기타 개인서비스직" "청소 및 기타 개인서비스직" ...
 $ age_20        : logi  FALSE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30        : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50        : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60        : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ age_70        : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증 : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ 지체장애_경증 : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증 : logi  FALSE TRUE TRUE TRUE FALSE TRUE ...
 $ 청각장애_경증 : logi  FALSE FALSE FALSE FALSE FALSE F

In [20]:
summary(data_filtered_subset)

                   취업직종대분류     target            age_20       
 청소 및 기타 개인서비스직:12681   Length:18912       Mode :logical  
 음식 서비스직            : 2197   Class :character   FALSE:13186    
 경호·경비직              : 1796   Mode  :character   TRUE :5726     
 영업·판매직              : 1150                                     
 스포츠·레크리에이션직    :  701                                     
 미용·예식 서비스직       :  387                                     
 (Other)                  :    0                                     
   age_30          age_40          age_50          age_60       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:16513     FALSE:16801     FALSE:15931     FALSE:14833    
 TRUE :2399      TRUE :2111      TRUE :2981      TRUE :4079     
                                                                
                                                                
                                                                
                                     

# 변동사항

In [21]:
data_filtered_subset$target <- factor(data_filtered_subset$target)

In [22]:
data_filtered_subset <- subset(data_filtered_subset, select = -c(취업직종대분류))

In [23]:
str(data_filtered_subset)

'data.frame':	18912 obs. of  46 variables:
 $ target       : Factor w/ 2 levels "서비스 및 판매 관련 직군",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ age_20       : logi  FALSE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30       : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50       : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60       : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ age_70       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증: logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ 지체장애_경증: logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증: logi  FALSE TRUE TRUE TRUE FALSE TRUE ...
 $ 청각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 정신장애_경증: logi  FALSE FALSE FALSE FAL

In [24]:
summary(data_filtered_subset)

                       target        age_20          age_30       
 서비스 및 판매 관련 직군 : 6231   Mode :logical   Mode :logical  
 청소 및 기타 개인서비스직:12681   FALSE:13186     FALSE:16513    
                                   TRUE :5726      TRUE :2399     
   age_40          age_50          age_60          age_70       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:16801     FALSE:15931     FALSE:14833     FALSE:17296    
 TRUE :2111      TRUE :2981      TRUE :4079      TRUE :1616     
 시각장애_중증   시각장애_경증   지체장애_중증   지체장애_경증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:18440     FALSE:17805     FALSE:17108     FALSE:14172    
 TRUE :472       TRUE :1107      TRUE :1804      TRUE :4740     
 지적장애_경증   지적장애_중증   청각장애_경증   청각장애_중증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:18902     FALSE:11288     FALSE:17914     FALSE:18019    
 TRUE :10        TRUE :7624      TRUE :998       TRUE :893      
 정신장애_경증   정신장애_중증   

In [25]:
lm(target~.,data_filtered_subset)


Call:
lm(formula = target ~ ., data = data_filtered_subset)

Coefficients:
      (Intercept)         age_20TRUE         age_30TRUE         age_40TRUE  
        1.2703368         -0.1233688         -0.0745017         -0.0051908  
       age_50TRUE         age_60TRUE         age_70TRUE  시각장애_중증TRUE  
        0.0647811          0.0319771                 NA         -0.0920188  
시각장애_경증TRUE  지체장애_중증TRUE  지체장애_경증TRUE  지적장애_경증TRUE  
       -0.1534462         -0.1893975         -0.1434939         -0.0091930  
지적장애_중증TRUE  청각장애_경증TRUE  청각장애_중증TRUE  정신장애_경증TRUE  
       -0.0008203         -0.0881415         -0.1081913          0.0750790  
정신장애_중증TRUE           경북TRUE           강원TRUE           전남TRUE  
               NA          0.4561317          0.3488945          0.4018853  
         충북TRUE           경기TRUE           인천TRUE           서울TRUE  
        0.4523008          0.4651239          0.2746349          0.3358814  
         울산TRUE           대전TRUE           부산TRUE           전북TRUE  
     

In [26]:
str(data_filtered_subset)
summary(data_filtered_subset)

'data.frame':	18912 obs. of  46 variables:
 $ target       : Factor w/ 2 levels "서비스 및 판매 관련 직군",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ age_20       : logi  FALSE TRUE FALSE TRUE FALSE TRUE ...
 $ age_30       : logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ age_40       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ age_50       : logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ age_60       : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ age_70       : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 시각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지체장애_중증: logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ 지체장애_경증: logi  TRUE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 지적장애_중증: logi  FALSE TRUE TRUE TRUE FALSE TRUE ...
 $ 청각장애_경증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 청각장애_중증: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ 정신장애_경증: logi  FALSE FALSE FALSE FAL

                       target        age_20          age_30       
 서비스 및 판매 관련 직군 : 6231   Mode :logical   Mode :logical  
 청소 및 기타 개인서비스직:12681   FALSE:13186     FALSE:16513    
                                   TRUE :5726      TRUE :2399     
   age_40          age_50          age_60          age_70       
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:16801     FALSE:15931     FALSE:14833     FALSE:17296    
 TRUE :2111      TRUE :2981      TRUE :4079      TRUE :1616     
 시각장애_중증   시각장애_경증   지체장애_중증   지체장애_경증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:18440     FALSE:17805     FALSE:17108     FALSE:14172    
 TRUE :472       TRUE :1107      TRUE :1804      TRUE :4740     
 지적장애_경증   지적장애_중증   청각장애_경증   청각장애_중증  
 Mode :logical   Mode :logical   Mode :logical   Mode :logical  
 FALSE:18902     FALSE:11288     FALSE:17914     FALSE:18019    
 TRUE :10        TRUE :7624      TRUE :998       TRUE :893      
 정신장애_경증   정신장애_중증   

# 시험준비

In [27]:
samp <- createDataPartition(data_filtered_subset$target, p=0.7, list=F)

#   학습용
data.tr <- data_filtered_subset[samp,]

#   검증용
data.test <- data_filtered_subset[-samp,]

In [28]:
#   KNN을 위한 Training Data 준비
data.training <- subset(data.tr, select = -target)
data.training.label <- data.tr$target

In [29]:
#   문제 및 정답 만들기
x   <- subset(data.test, select = -target)
y   <- data.test$target


# 학습시키기

In [30]:
#  랜덤포레스트 학습 시키기
rf <- randomForest(target ~., data.tr, ntree=500)

In [31]:
# SVM 학습시키기
modelsvm <- svm(target~.,data.tr)

In [32]:
# KNN 학습시키기 3,5,7,
# knnmodel <- knn(train=data.training, test=x,cl=data.training.label,k=3)

In [33]:
# nnet 학습시키기
modelnnet <- nnet(target~.,data.tr,size =20)

# weights:  941
initial  value 11218.965079 
iter  10 value 7831.480453
iter  20 value 7613.021929
iter  30 value 7376.522062
iter  40 value 7213.529973
iter  50 value 7056.974015
iter  60 value 6904.945566
iter  70 value 6780.262712
iter  80 value 6693.172064
iter  90 value 6627.387969
iter 100 value 6552.578854
final  value 6552.578854 
stopped after 100 iterations


# 검증하기

In [34]:
#  Random Forest 검증하기
predrf <- predict(rf, x)

In [35]:
#  SVM 검증하기
predmodelsvm <- predict(modelsvm, x)

In [36]:
# KNN 검증하기
# mean(knnmodel==y) 

In [37]:
# nnet 검증하기
prednnet = predict(modelnnet, x,type = "class")    # class : 객관식, raw : 주관식

# 예측력 확인

In [38]:
#  RF 예측력
mean(predrf == y)

[1] 0.7057994

In [39]:
#  SVM 예측력
mean(predmodelsvm == y)

[1] 0.6705447

In [40]:
#  KNN 예측력
# mean(knnmodel == y)

In [41]:
#  nnet 예측력
mean(prednnet == y)

[1] 0.6878195

# 오분류표

In [42]:
#  Random Forest 오분류표
table(predrf,y)

                           y
predrf                      서비스 및 판매 관련 직군 청소 및 기타 개인서비스직
  서비스 및 판매 관련 직군                       576                       376
  청소 및 기타 개인서비스직                     1293                      3428

In [43]:
#  SVM 오분류표
table(predmodelsvm,y)

                           y
predmodelsvm                서비스 및 판매 관련 직군 청소 및 기타 개인서비스직
  서비스 및 판매 관련 직군                         0                         0
  청소 및 기타 개인서비스직                     1869                      3804

In [44]:
#  KNN 오분류표
# table(y, knnmodel)

In [45]:
# nnet 오분류표
table(prednnet,y)

                           y
prednnet                    서비스 및 판매 관련 직군 청소 및 기타 개인서비스직
  서비스 및 판매 관련 직군                       773                       675
  청소 및 기타 개인서비스직                     1096                      3129

In [46]:
# 새로운 데이터에 대한 예측
predict(rf, (list(age_20=FALSE,age_30=FALSE,age_40=FALSE,age_50=FALSE,age_60=TRUE,age_70=FALSE,
            시각장애_중증=TRUE,시각장애_경증=FALSE,지체장애_경증=FALSE,지체장애_중증=FALSE,지적장애_경증=FALSE,지적장애_중증=FALSE,
            청각장애_경증=FALSE,청각장애_중증=FALSE,정신장애_경증=FALSE,정신장애_중증=FALSE,
            경북=TRUE,강원=FALSE,전남=FALSE,충북=FALSE,경기=FALSE,인천=FALSE,서울=FALSE,울산=FALSE,대전=FALSE,부산=FALSE,
            전북=FALSE,광주=FALSE,충남=FALSE,대구=FALSE,경남=FALSE,제주=FALSE,세종=FALSE,경북=FALSE,
            Jan=TRUE,Feb=FALSE,Mar=FALSE,Apr=FALSE,May=FALSE,Jun=FALSE,Jul=FALSE,Aug=FALSE,Sep=FALSE,
            Oct=FALSE,Nov=FALSE,Dec=FALSE)), type = "prob")

# pred_prob <- predict(rf, new_data, type = "prob")
# pred_prob <- predict(rf, new_data, type = "prob")

,서비스 및 판매 관련 직군,청소 및 기타 개인서비스직
1,0,1
